In [1]:
# Install AutoGen
!pip install "autogen-agentchat==0.4.0.dev13" "autogen-ext[openai,azure]==0.4.0.dev13"

In [ ]:
# Import required packages
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from datetime import datetime, timedelta
import json
import asyncio


In [ ]:

# This tool manages a simple task list stored in memory
tasks = []

async def add_task(title: str, due_date: str, priority: str = "medium") -> dict:
    """
    Adds a new task to our task list. Returns the created task for confirmation.
    The due_date should be in YYYY-MM-DD format.
    """
    task = {
        "id": len(tasks) + 1,
        "title": title,
        "due_date": due_date,
        "priority": priority,
        "created_at": datetime.now().strftime("%Y-%m-%d"),
        "status": "pending"
    }
    tasks.append(task)
    return task

async def get_tasks(status: str = None) -> list:
    """
    Retrieves tasks, optionally filtered by status.
    Demonstrates how to handle query parameters.
    """
    if status:
        return [task for task in tasks if task["status"] == status]
    return tasks

async def mark_complete(task_id: int) -> dict:
    """
    Marks a task as complete. Shows how to update existing data.
    """
    for task in tasks:
        if task["id"] == task_id:
            task["status"] = "completed"
            return task
    return {"error": "Task not found"}


In [ ]:

async def main() -> None:
    # Create an assistant specialized in task management
    task_manager = AssistantAgent(
        name="task_manager",
        model_client=OpenAIChatCompletionClient(
            model="gpt-4o",
            api_key= "Your-OpenAI-API-Key"
        ),
        system_message="""You are a helpful task management assistant.
        Help users organize their tasks, set due dates, and track progress.
        Always confirm actions and provide clear status updates.""",
        tools=[add_task, get_tasks, mark_complete],
    )

    # Create a team with our task manager agent
    agent_team = RoundRobinGroupChat([task_manager], max_turns=2)

    # Start the interactive loop
    print("Task Manager Assistant (type 'exit' to leave)")
    print("Try: 'Add a task to review the quarterly report by next Friday'")
    print("Or: 'Show my pending tasks'")

    while True:
        user_input = input("\nWhat would you like to do? ")
        if user_input.strip().lower() == "exit":
            break
        stream = agent_team.run_stream(task=user_input)
        await Console(stream)

# Run with: asyncio.run(main())

In [ ]:
await main()

Task Manager Assistant (type 'exit' to leave)
Try: 'Add a task to review the quarterly report by next Friday'
Or: 'Show my pending tasks'

What would you like to do? show pending
---------- user ----------
show pending
---------- task_manager ----------
[FunctionCall(id='call_DS0JO50ND7FviYMRxONIZSqs', arguments='{"status":"pending"}', name='get_tasks')]
[Prompt tokens: 212, Completion tokens: 15]
---------- task_manager ----------
[FunctionExecutionResult(content='[]', call_id='call_DS0JO50ND7FviYMRxONIZSqs')]
---------- task_manager ----------
[]
---------- task_manager ----------
It seems that there are no pending tasks at the moment. If you have any new tasks to add, please let me know!
[Prompt tokens: 238, Completion tokens: 27]
---------- Summary ----------
Number of messages: 5
Finish reason: Maximum number of turns 2 reached.
Total prompt tokens: 450
Total completion tokens: 42
Duration: 3.04 seconds

What would you like to do? add a task to buy a toy
---------- user ----------